# **Genius Scraper**


### Using Genius to scrape data on the title, artist and song lyrics

---

### **Set-up ⚙️**

Import necessary packages

*⚠️ Note: Do not run this more than once. Restart the kernel before running this code chunk.*

In [1]:
import json
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import os
os.chdir(os.path.expanduser("../"))                 # change directory to root of project

from dees_package.genius_functions import *         # imports custom functions for genius scraping

Check that we are in the correct current working directory

*⚠️ Note: We should be in the root of the project folder*

In [2]:
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\soong\Documents\LSE_Acads\AT_23.24\DS105A\final_project\ds105a-project-dees-nuts


Open JSON file containing credentials

*⚠️ Note: Your credentials should be stored in a file titled `credentials.json` and stored in the root of the project folder*

In [ ]:
credentials_file_path = './credentials.json'

with open(credentials_file_path, 'r') as f:
    credentials = json.load(f)

Initialise a new session

In [ ]:
my_session = requests.Session()

---

### **Data Scraping 🔍**

Import and clean YouTube data into dataframe

In [ ]:
raw_df = pd.read_csv('./data/actual_youtube_data_50.csv')
cleaned_df = raw_df.drop(['Unnamed: 0', 'video_id', 'channel_title', 'channel_id', 'description'], axis=1)
cleaned_df = cleaned_df.rename(columns={'title': 'video_title'})
cleaned_df['video_title'] = cleaned_df['video_title'].str.replace(r'\(.*\)|\s+ft\..*', '', regex=True)

Create dataframe containing scraped data for the title, artist and URL

In [ ]:
scraped_df = pd.DataFrame([search_genius(q, credentials['client_access_token']) for q in tqdm(cleaned_df['video_title'])])

Add lyrics to dataframe

In [ ]:
scraped_df['lyrics'] = scraped_df.apply(lambda row: scrape_lyrics(my_session, row['URL']) if row['URL'] else '', axis=1)

Clean dataframe by removing rows with empty lyrics

In [ ]:
scraped_df.dropna(subset=['lyrics'], inplace=True)
df = scraped_df[scraped_df.astype(bool).any(axis=1)]

Save dataframe to csv

In [ ]:
scraped_df.to_csv('./data/data_with_lyrics.csv')